In [0]:
spark.sql("USE CATALOG hp_az_catalog") 
spark.sql("USE SCHEMA hp_iris_ml")

In [0]:
%pip install mlflow=='3.4.0'
dbutils.library.restartPython()

In [0]:
import numpy
import pandas as pd
import mlflow
from sklearn import datasets

In [0]:
iris_samples = datasets.load_iris(as_frame=True)
df_samples = pd.DataFrame(data = iris_samples['data'], columns = iris_samples['feature_names'])
df_samples.columns = df_samples.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df_samples['species'] = iris_samples.target.astype(int)
df_samples.head()

In [0]:
model_info = mlflow.pyfunc.load_model(model_uri)
print(model_info.metadata.get_input_schema())

In [0]:
eval_data = df_samples.rename(columns={
    "sepal_length_cm": "sepal length (cm)",
    "sepal_width_cm": "sepal width (cm)",
    "petal_length_cm": "petal length (cm)",
    "petal_width_cm": "petal width (cm)"
})

target = "species"
model_type = "classifier"
model_name = "irisclassifierdemo"
model_version = "5"
model_uri = f"models:/{model_name}/{model_version}"

with mlflow.start_run(run_name="evaluation") as run:
    mlflow.models.evaluate(
        model=model_uri,
        data=eval_data,
        targets="species",
        model_type=model_type
    )
